# Read Downloaded Datasets

To do this project, I have downloaded essential dataset that i will use to analyze from Fred.

__Objectives__

The primary purpose of this notebook is to parse the dataset in mbox file into a format that will allow me to analyze the words in the headline for further sentimental analysis. 


In [70]:
import mailbox
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import string
import re
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [71]:
# The directory used to store the raw datasets.
datasets_dir = '../../FYP/Raw_dataset/'
# The path to the dataset that is produced by this notebook.
headlines_dir = '../../FYP/Raw_dataset/1000_news_headlines.csv'

In [72]:
# Read mbox file
mbox_file_path = datasets_dir + 'GoogleRitual.mbox'
mbox = mailbox.mbox(mbox_file_path)

By printing extracted subject value we can see that those subjects are not helpful so i am going to extract data from its body content.

In [73]:
# Extract subjects using list comprehension, extract the value of Subject field if Subject is not empty"
subjects = [message['Subject'] for message in mbox if message['Subject']]

In [74]:
subjects

['Google Alert - ritual',
 'Google Alert - ritual',
 'Google Alert - ritual',
 'Google Alert - ritual',
 'Google Alert - ritual',
 'Google Alert - ritual',
 'Google Alert - ritual',
 'Google Alert - ritual',
 'Google Alert - ritual',
 'Google Alert - ritual',
 'Google Alert - ritual',
 'Google Alert - ritual',
 'Google Alert - ritual',
 'Google Alert - ritual',
 'Google Alert - ritual',
 'Google Alert - ritual',
 'Google Alert - ritual',
 'Google Alert - ritual',
 'Google Alert - ritual',
 'Google Alert - ritual',
 'Google Alert - ritual',
 'Google Alert - ritual',
 'Google Alert - ritual',
 'Google Alert - ritual',
 'Google Alert - ritual',
 'Google Alert - ritual',
 'Google Alert - ritual',
 'Google Alert - ritual',
 'Google Alert - ritual',
 'Google Alert - ritual',
 'Google Alert - ritual',
 'Google Alert - ritual',
 'Google Alert - ritual',
 'Google Alert - ritual',
 'Google Alert - ritual',
 'Google Alert - ritual',
 'Google Alert - ritual',
 'Google Alert - ritual',
 'Google Ale

In [75]:
Date = []
contents = []
count = 0
for message in mbox:
#     if count ==1:
#         break
    # append date in the email into Date list, used list slice to only keep necessary data.
    Date.append(message['date'][5:16])
    # retrieves the content of the email
    payload = message.get_payload()
#     count +=1
    # If the message is multipart
    if message.is_multipart():
        # iterate through each part of the email
        for part in payload:
            # Check if the content type is text (this will filter out attachments, images, etc.)
            if part.get_content_type() in ['text/plain']:
                decoded_part = part.get_payload(decode=True)
                if decoded_part:
                    contents.append(decoded_part.decode('utf-8', 'ignore'))
    else:
        if payload:
            contents.append(payload)

In [76]:
Date

['13 Feb 2021',
 '01 Jan 2022',
 '25 Sep 2021',
 '25 Dec 2021',
 '03 Apr 2021',
 '24 Oct 2020',
 '06 Apr 2019',
 '15 Sep 2018',
 '29 Jun 2019',
 '27 Jun 2020',
 '14 Nov 2020',
 '07 Sep 2019',
 '14 Jul 2018',
 '04 Dec 2021',
 '21 Aug 2021',
 '21 Nov 2020',
 '04 Aug 2018',
 '29 Jan 2022',
 '28 Aug 2021',
 '08 Jun 2019',
 '05 Feb 2022',
 '16 May 2020',
 '20 Jul 2019',
 '04 Jul 2020',
 '17 Oct 2020',
 '24 Mar 2018',
 '20 Nov 2021',
 '13 Oct 2018',
 '28 Sep 2019',
 '12 May 2018',
 '28 Apr 2018',
 '02 Jul 2022',
 '24 Jul 2021',
 '28 Dec 2019',
 '20 Oct 2018',
 '20 Apr 2019',
 '18 Apr 2020',
 '15 Aug 2020',
 '10 Oct 2020',
 '11 Apr 2020',
 '29 Aug 2020',
 '24 Apr 2021',
 '07 Dec 2019',
 '16 Jan 2021',
 '18 Jun 2022',
 '21 Sep 2019',
 '27 Apr 2019',
 '10 Jul 2021',
 '22 Feb 2020',
 '20 Aug 2022',
 '13 Aug 2022',
 '28 Nov 2020',
 '15 Jun 2019',
 '27 Oct 2018',
 '17 Mar 2018',
 '01 Dec 2018',
 '14 Mar 2020',
 '14 Dec 2019',
 '30 Oct 2021',
 '01 Sep 2018',
 '04 Jun 2022',
 '23 Mar 2019',
 '29 Dec

By observing the body content we can find the pattern that for each headline \r\n\r\n appear before it and \r\n appear at the end, for each of line there is 75 char max limitation which possibly would cause either headline or description span multilines.

One of challenge would be extracting this body content into three essential field: headline, pubulisher, tag text, As Each of field is seperated by the same delimiter \r\n, as well as the headline and tag text can span mulitilines by the same delimiter \r\n so it is hard to determine whether the next line is continuation of previous or not.
In order to extract the headline fully  I have to make the following assumption as my heuristic approach:

1. If the length of first line for headline + the length of first word in the next line after headline is not greater than 75 we can easily conclude that this headline does not span multilines.

2. If the first case is not true, then if the first line for headline is 75 or less than and its next line start with lowercase letter then it is probably a continuation of the previous line.


In [77]:
contents[0]

'=== News - 10 new results for [ritual] ===\r\n\r\nPope resumes Sunday ritual of greeting public\r\nDaily Sabah\r\nPope Francis on Sunday delivered the Angelus prayer and greeted the public\r\nin St. Peter\'s Square, resuming the Sunday noon tradition in the Vatican ...\r\n<https://www.google.com/url?rct=j&sa=t&url=https://www.dailysabah.com/world/pope-resumes-sunday-ritual-of-greeting-public/news&ct=ga&cd=CAEYACoTNTMzOTgzNTQ4ODI1Njc2OTMyOTIZMThlOTc2YWJlZWU0YjcyMTppZTplbjpJRQ&usg=AFQjCNFgRsDmDgH1GgUKsn96jgTHVIT2pA>\r\n\r\nAP PHOTOS: Afro-Brazilian religion ritual before Carnival\r\nThe Philadelphia Inquirer\r\nThe faithful of Umbanda, a religion that was born in Brazil, always perform\r\nrituals of spiritual protection as part of the pre-Carnival traditions. A\r\nspiritual ...\r\n<https://www.google.com/url?rct=j&sa=t&url=https://www.inquirer.com/wires/ap/ap-photos-afro-brazilian-religion-ritual-before-carnival-20210207.html&ct=ga&cd=CAEYASoTNTMzOTgzNTQ4ODI1Njc2OTMyOTIZMThlOTc2YWJlZWU0

In [78]:
def find_word_length_at_index(input_text, start_index):
    # Initialize a variable to store the length of the word
    word_length = 0

    # Iterate through the string starting from the given index
    index = start_index
    while index < len(input_text) and input_text[index] != ' ':
        word_length += 1
        index += 1
    return word_length

In [79]:
def FindTag_txt(lines, start_index):
    i = 0
    tag_text = ''
    google_url = 'https'
    while True:
        current_index = start_index + i
        if google_url not in lines[current_index + 1]:
            i = i + 1
            tag_text = tag_text + lines[current_index]
        else:
            tag_text = tag_text + lines[current_index]
            break
    return tag_text

In [80]:
date = []
headlines = []
authors = []
tag_texts = []
line_char_limitation = 75
for count, content in enumerate(contents):
    # For each email content in the list of contents we use regular expression to find the pattern in the text.
    matches = re.findall(r'\r\n\r\n(.*?)\r\n', content)
    lines = content.split('\r\n')
    # For each headline we do not know whether its next line (seperated by /r/n) is the continuation or not, so we use the approach mentioned previously.
    for match in matches:
        headline_index = lines.index(match)
        l = len(match)
        first_word_len = find_word_length_at_index(lines[headline_index+1],0)
        total_len = first_word_len + l
        # if total length is > 75 then we know the headline possbily span multilines.
        if total_len >= line_char_limitation:
            # if the next line start with capital letter then it is likely a new line for author.
            if lines[headline_index+1][0].isupper():
                if "ritual" in match.lower():
                    headlines.append(match)
                    date.append(Date[count])
                    author_index = headline_index + 1
                    # Handle the case that some of author is missing so the next line of headline will be tag text.
                    if len(lines[author_index]) > 60:
                        authors.append(None)
                        tag_texts.append(FindTag_txt(lines, author_index))
                    # No need to handle that case.
                    else:
                        authors.append(lines[author_index])
                        tag_texts.append(FindTag_txt(lines, author_index + 1))
                    
            else:
                # if the next line start with lowercase it is likely a continuation of headline.
                Full_headline = lines[headline_index] + " " + lines[headline_index+1]
                if "ritual" in Full_headline.lower():
                    headlines.append(Full_headline)
                    date.append(Date[count])
                    author_index = headline_index + 2
                    # Handle the case that some of author is missing so the next line of headline will be tag text.
                    if len(lines[author_index]) > 60:
                        authors.append(None)
                        tag_texts.append(FindTag_txt(lines, author_index))
                    # No need to handle that case.
                    else:
                        authors.append(lines[author_index])
                        tag_texts.append(FindTag_txt(lines, author_index + 1))
                    
            
        else:
            # headline length + the length of first word in the next line < 75 then we know headline does not span multilines so we append to list.
            if "ritual" in match.lower():
                headlines.append(match)
                date.append(Date[count])
                author_index = headline_index + 1
                # Handle the case that some of author is missing so the next line of headline will be tag text.
                if len(lines[author_index]) > 60:
                    authors.append(None)
                    tag_texts.append(FindTag_txt(lines, author_index))
                # No need to handle that case.
                else:
                    authors.append(lines[author_index])
                    tag_texts.append(FindTag_txt(lines, author_index + 1))
                
# # use list comprehension to filter the original list to generate the list that for each string in the list contain ritual as substring.
# filtered_headlines = [item for item in headlines if "ritual" in item.lower()]

In [81]:
headlines

['Pope resumes Sunday ritual of greeting public',
 'AP PHOTOS: Afro-Brazilian religion ritual before Carnival',
 'Ash Wednesday ritual will change due to COVID concerns',
 'Bloodstained: Ritual of the Night Has Been Hiding a Secret, Extra-Difficult 1986 Mode',
 "Make Rituals A Part Of Your Company Roadmap. Here's How",
 "Larry Johnson Believes The Weeknd's Half-time Show Was A 'Satanic Ritual'",
 'Column: Finding New Ground in Ritual',
 'Amazon.com: Daily Ritual: Supersoft Terry',
 'Destiny 2 on Twitter: "Engage in ritual combat against Caiatl\'s top chosen warriors at the ...',
 '5 Simple Rituals To Help You Reflect On 2021 & Plan For 2022 -',
 'Resurrection ritual in Bangkok - YouTube',
 'Linking arms while singing Auld Lang Syne was a parting ritual of the',
 'Peruvians engage in guinea-pig-rubbing ritual to prepare for 2022 - YouTube',
 "Hamilton Morris Attends an African Psychedelic Ritual | HAMILTON'S",
 'In pictures: Whirling dervish ritual honours Rumi, the Sufi mystic poet -',

In [82]:
authors

['Daily Sabah',
 'The Philadelphia Inquirer',
 'masslive.com',
 'IGN',
 'Forbes',
 'Republic World',
 'The Wild Hunt',
 None,
 None,
 'MindBodyGreen',
 'YouTube',
 'Freemasons, expert reveals',
 'YouTube',
 'PHARMACOPEIA - YouTube',
 'The Indian Express',
 'Lowell Sun',
 'Bears Wire - USA Today',
 'Tribune Online',
 None,
 'Engine',
 None,
 'The Irish Times',
 'NPR',
 'America Magazine',
 'Crime | The Guardian',
 'The Washington Post',
 'Archaeology',
 'Inc.com',
 'The Conversation US',
 'Health',
 'Shows - ScienceAlert',
 'MindBodyGreen',
 'The New York Times',
 'The New York Times',
 'Daily Mail',
 'Woman & Home',
 'Washington Post',
 'AP News',
 'Euronews',
 'Happened, Says Study',
 'The Guardian',
 'The Irish Sun',
 'Mongabay-India',
 'DFNIonline.com',
 'ecns',
 'The Moodie Davitt Report',
 'KPVI News 6',
 'KCRW',
 'ccxmedia.org',
 'Yahoo Finance',
 None,
 'Events - William & Mary',
 'New Scientist',
 'The Irish Sun',
 'The Guardian',
 'The Times of India Blog',
 'Phys.Org',
 'The 

In [83]:
tag_texts

["Pope Francis on Sunday delivered the Angelus prayer and greeted the publicin St. Peter's Square, resuming the Sunday noon tradition in the Vatican ...",
 'The faithful of Umbanda, a religion that was born in Brazil, always performrituals of spiritual protection as part of the pre-Carnival traditions. Aspiritual ...',
 "The Ash Wednesday ritual of a mark on the forehead in the shape of a cross— the day's reminder of humanity's mortal state and start of Lent — will be...",
 'Bloodstained: Ritual of the Night got an update adding a Classic Mode lastmonth, but IGN can reveal that the update added a second, hidden mode ...',
 "That's a ritual. I asked Erica why startup CEOs – who very often havetrouble celebrating milestones because of their intense drive – shouldintentionally ...",
 "Former Chiefs star Larry Johnson has gone viral again after calling TheWeeknd's Super Bowl 2021 half-time performance a 'satanic ritual'. WrittenBy.",
 'Ritual forms a powerful bond during each and every ste

In [84]:
date

['13 Feb 2021',
 '13 Feb 2021',
 '13 Feb 2021',
 '13 Feb 2021',
 '13 Feb 2021',
 '13 Feb 2021',
 '13 Feb 2021',
 '13 Feb 2021',
 '13 Feb 2021',
 '01 Jan 2022',
 '01 Jan 2022',
 '01 Jan 2022',
 '01 Jan 2022',
 '01 Jan 2022',
 '01 Jan 2022',
 '01 Jan 2022',
 '01 Jan 2022',
 '01 Jan 2022',
 '01 Jan 2022',
 '01 Jan 2022',
 '25 Sep 2021',
 '25 Sep 2021',
 '25 Sep 2021',
 '25 Sep 2021',
 '25 Sep 2021',
 '25 Sep 2021',
 '25 Sep 2021',
 '25 Sep 2021',
 '25 Sep 2021',
 '25 Sep 2021',
 '25 Dec 2021',
 '25 Dec 2021',
 '25 Dec 2021',
 '25 Dec 2021',
 '25 Dec 2021',
 '25 Dec 2021',
 '25 Dec 2021',
 '25 Dec 2021',
 '25 Dec 2021',
 '25 Dec 2021',
 '03 Apr 2021',
 '03 Apr 2021',
 '03 Apr 2021',
 '03 Apr 2021',
 '03 Apr 2021',
 '03 Apr 2021',
 '03 Apr 2021',
 '03 Apr 2021',
 '03 Apr 2021',
 '03 Apr 2021',
 '03 Apr 2021',
 '03 Apr 2021',
 '24 Oct 2020',
 '24 Oct 2020',
 '24 Oct 2020',
 '24 Oct 2020',
 '24 Oct 2020',
 '24 Oct 2020',
 '24 Oct 2020',
 '24 Oct 2020',
 '24 Oct 2020',
 '24 Oct 2020',
 '24 Oct

In [85]:
# Split the content into lines
# lines = content.split('\r\n')
# lines
# # Filter out lines
# # we're considering lines that don't start with '<' (links) and '===' (section dividers) as headlines, author and description
# hds = [line for line in lines if not line.startswith('<') and not line.startswith('===') and not line.startswith('- - -') and line.strip() != '']

# hds

In [86]:
data = {"Date":date,"Headlines":headlines,"Author":authors,"Tag Text":tag_texts}

In [87]:
df = pd.DataFrame(data)

In [88]:
df['Date'] = pd.to_datetime(df['Date']).dt.date

In [89]:
df.shape

(2205, 4)

In [90]:
df.head()

,Date,Headlines,Author,Tag Text
0,2021-02-13,Pope resumes Sunday ritual of greeting public,Daily Sabah,Pope Francis on Sunday delivered the Angelus p...
1,2021-02-13,AP PHOTOS: Afro-Brazilian religion ritual befo...,The Philadelphia Inquirer,"The faithful of Umbanda, a religion that was b..."
2,2021-02-13,Ash Wednesday ritual will change due to COVID ...,masslive.com,The Ash Wednesday ritual of a mark on the fore...
3,2021-02-13,Bloodstained: Ritual of the Night Has Been Hid...,IGN,Bloodstained: Ritual of the Night got an updat...
4,2021-02-13,Make Rituals A Part Of Your Company Roadmap. H...,Forbes,That's a ritual. I asked Erica why startup CEO...


In [91]:
print('write the dataFrame of extracted data to news_headlines.csv. (%s, %s.)' % df.shape)
df.to_csv(headlines_dir, index=False)
df.shape, df.columns

write the dataFrame of extracted data to news_headlines.csv. (2205, 4.)


((2205, 4), Index(['Date', 'Headlines', 'Author', 'Tag Text'], dtype='object'))